# Generation of λ-Nets for I-Net training

## 0: Preparation

In [1]:
##############DO NOT CHANGE###################

# Importing own helper library
import sys
sys.path.insert(0,'../_baselib')
import general_helper as gh
import polynom_helper as ph
import lambdanet_helper as lh

# Third-party imports
import ttg
from itertools import product       # forms cartesian products
from tqdm.notebook import tqdm
from more_itertools import random_product 
import pickle
import numpy as np
import pandas as pd
import timeit
import matplotlib.pyplot as plt 
import time
import math
import random 
import os
import logging
import warnings

import seaborn as sns
from scipy.signal import argrelextrema

import shutil

from joblib import Parallel, delayed

from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score, mean_absolute_error, r2_score
from similaritymeasures import frechet_dist, area_between_two_curves, dtw
from IPython.display import Image

from livelossplot.keras import PlotLossesCallback

import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# sympy for Polynomial support
from sympy.polys import monomials
from sympy.polys.orderings import monomial_key
from sympy import symbols

In [2]:
# Static settings & directory preparation

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

logging.getLogger('tensorflow').disabled = True

sns.set_style("darkgrid")

np.set_printoptions(suppress=True)

warnings.filterwarnings("ignore", category=FutureWarning)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

#keras.backend.set_epsilon(10e-3)

if int(tf.__version__[0]) >= 2:
    tf.random.set_seed(RANDOM_SEED)
else:
    tf.set_random_seed(RANDOM_SEED)

## 1: Specification of Experiment Settings

In [3]:
# parse settings from a configuration file ('config.ini')

config_path = 'config.ini'

try:
    config = gh.parse_config(config_path)
    locals().update(config)
    
except Exception as e:
    print('Parsing not possible due to Exception:')
    print(e)
    print('\nContinue with manual specification.')

In [4]:
# manual override of config and/or additional variable specification

#d = 3  
#n = 4
#sparsity = 0.0
#a_max = 10 
#a_min = -10
#a_step = 0.1
#x_max = 1 
#x_min = -1
#x_step = 0.01
#lambda_dataset_size = 100 
#interpretation_dataset_size = 5000
#same_training_all_polynomials = True

n_jobs = -3

# - lambda net specifications 
#batch_size = 64
#epochs = 200
#dropout = 0.0
#optimizer='SGD'
#each_epochs_save = 5

# how many lambda nets should be generated (must be smaller or equal to interpretation_dataset_size)
number_of_lambda_nets = interpretation_dataset_size

# ------------------------------------------------------------------

In [8]:
##############DO NOT CHANGE###################

# set derived attributes
variables = 'abcdefghijklmnopqrstuvwxyz'[:n]
if same_training_all_polynomials: 
    training_string = '_same'
else: 
    training_string = '_diverse'

if number_of_lambda_nets < interpretation_dataset_size:
    generate_subset = True
    data_size = number_of_lambda_nets
else:
    generate_subset = False
    data_size = interpretation_dataset_size   

seed_method = True
shuffle = True

lambda_network_layers = [5 * gh.nCr(n+d, d)]
layers_str = ''.join([str(neurons) + '-' for neurons in lambda_network_layers])

advanced_metric_dataset_size = 10#200

if seed_method and shuffle:
    seed_shuffle_string = '_shuffleSeedMethod'
elif not seed_method and shuffle:
    seed_shuffle_string = '_shuffleNoSeedMethod'
elif seed_method and not shuffle:
    seed_shuffle_string = '_noShuffleSeedMethod'
elif not seed_method and not shuffle:
    seed_shuffle_string = '_noShuffleNoSeedMethod'
    
use_gpu = False
if use_gpu:
    gpu_numbers = '0'
else:
    gpu_numbers = ''
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_numbers
    
structure = '_' + layers_str + str(epochs) + 'e' + str(batch_size) + 'b_' + optimizer 
    
filename = seed_shuffle_string + '_' + str(RANDOM_SEED) + structure

In [11]:
# inspect most important settings
print('Variables: ' + str(n) + ' (' + variables + ')')
print('Degree: ' + str(d))
print('Sparsity: ' + str(sparsity*100) + '%') 
print('Lambda-Net Dataset Size: ' + str(lambda_dataset_size))
print('I-Net Dataset Size: ' + str(interpretation_dataset_size))
print('Number of Lambda-Nets to Train: ' + str(number_of_lambda_nets))
      
print('Coefficient Range: ' + '[' + str(a_min) + ', ' + str(a_max) + ']')
print('Variable Range: ' + '[' + str(x_min) + ', ' + str(x_max) + ']')

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Variables: 4 (abcd)
Degree: 3
Sparsity: 0.0%
Lambda-Net Dataset Size: 100
I-Net Dataset Size: 5000
Number of Lambda-Nets to Train: 5000
Coefficient Range: [-10.0, 10.0]
Variable Range: [-1.0, 1.0]
Num GPUs Available:  0
Num XLA-GPUs Available:  0


In [12]:
#def calcualate_function_with_data(coefficient_list, variable_values):
#    
#    global list_of_monomial_identifiers
#    
#    result = 0    
#    for coefficient_value, coefficient_multipliers in zip(coefficient_list, list_of_monomial_identifiers):
#        partial_results = [variable_value**int(coefficient_multiplier) for coefficient_multiplier, variable_value in zip(coefficient_multipliers, variable_values)]
#        
#        result += coefficient_value * reduce(lambda x, y: x*y, partial_results)
#
#    return result, variable_values
# 
#def calculate_function_values_from_polynomial(true_value_test, evaluation_dataset):
#
#    if isinstance(true_value_test, pd.DataFrame):
#        true_value_test = true_value_test.values
#        
#    true_value_fv = []
#    true_value_coeff = []
#        
#    for evaluation in evaluation_dataset:
#        true_function_value, true_coeff = calcualate_function_with_data(true_value_test, evaluation)
#       
#        true_value_fv.append(true_function_value) 
#        true_value_coeff.append(true_coeff)
#        
#    return np.array(true_value_coeff), np.array(true_value_fv)


## 2: Data Import (from Polynomial Generation)

In [13]:
# Csv imports
path_polynomials_df = './data/saved_polynomial_lists/polynomials_df' + str(interpretation_dataset_size) + '_variables_' + str(n) +  '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '.csv'
polynomials_df = gh.import_csv(path_polynomials_df)

# Pickle imports
path_polynomials_poly = './data/saved_polynomial_lists/polynomials_poly' + str(interpretation_dataset_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step) + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '.pkl'
polynomials_poly = gh.import_pickle(path_polynomials_poly)

path_x_data = './data/saved_polynomial_lists/x_data' + str(interpretation_dataset_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step) + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '.pkl'
x_data = gh.import_pickle(path_x_data)

path_y_data = './data/saved_polynomial_lists/y_data' + str(interpretation_dataset_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step) + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '.pkl'
y_data = gh.import_pickle(path_y_data)
    
if generate_subset:
    polynomials_df = polynomials_df.sample(n=number_of_lambda_nets, random_state=RANDOM_SEED)
    random.seed(RANDOM_SEED)
    x_data = random.sample(x_data, number_of_lambda_nets)
    random.seed(RANDOM_SEED)
    y_data = random.sample(y_data, number_of_lambda_nets)
    random.seed(RANDOM_SEED)


In [14]:
# inspect first polynomial of x_data
print(len(x_data[0]))
x_data[0]

100


array([[-0.39,  0.98,  0.6 , -0.97],
       [ 0.35,  0.14,  0.61, -0.27],
       [-0.65,  0.56, -0.33, -0.66],
       [-0.73, -0.79, -0.43,  0.16],
       [-0.5 , -0.75,  0.33, -0.77],
       [-0.33, -0.55,  0.98,  0.34],
       [-0.64, -0.33,  0.23,  0.5 ],
       [ 0.59, -0.47,  0.82, -0.16],
       [ 0.38, -0.21, -0.61,  0.48],
       [ 0.96, -0.54,  0.22,  0.26],
       [ 0.17, -0.66,  0.74, -0.53],
       [-0.75, -0.72,  0.86, -0.86],
       [ 0.98, -0.2 ,  0.  ,  0.35],
       [-0.6 ,  0.28,  0.59, -0.98],
       [ 0.6 , -0.21, -0.73,  0.88],
       [ 0.84, -0.43, -0.77,  0.22],
       [-0.01, -0.03, -0.27, -0.51],
       [ 0.53,  0.76,  0.99, -0.89],
       [ 0.69,  0.28,  0.7 , -0.54],
       [ 0.88,  0.54, -0.19, -0.37],
       [ 0.77,  0.77, -0.25,  0.69],
       [ 0.7 ,  0.16,  0.08,  0.76],
       [-0.21,  0.3 ,  0.9 , -0.89],
       [-0.92,  0.76,  0.21,  0.16],
       [ 0.8 , -0.55, -0.38, -0.89],
       [ 0.91, -0.15,  0.67,  0.04],
       [-0.78, -0.2 ,  0.43, -0.18],
 

In [17]:
# inspect first polynomial of y_data
print(len(y_data[0]))
y_data[0]

100


array([  6.5466964,  14.1552117,   6.2386106,  -5.1563538,  -0.3081001,
        14.2110167,   2.7051367,   9.8477365,  13.3632263,  -3.7052808,
         6.8799222,  21.431129 ,   5.5486424,  10.2749953,  14.4934895,
        12.0238791,   4.7583472,  36.0919691,  21.6437686,  14.6957146,
        14.033763 ,   7.48636  ,  20.1434354,  22.4756171,   2.4145583,
        10.1207996,   8.0853943,  17.3275651,   4.8414354,  13.8312942,
        10.0359621,   8.5623464,  11.232144 ,   8.3449984,   8.26664  ,
        14.9745371,  19.6612157,   9.5912396,  11.5108923,  12.6079121,
        31.8877648,  19.0505869,   5.1019445,  21.3153018,  10.3172595,
        17.2387127,  12.0904572,  22.7792788,  -2.8126045,  19.4617989,
        -4.7880812,   8.1800906,  10.9686977,  21.3805084,  10.5591725,
         5.7823363,  12.4230728,  10.5239151,  -5.3532647,   7.8290411,
         5.3607768,  11.6056111,  11.0576213,   5.9079895,   4.3057739,
        10.1469917,  10.3934912,  23.213084 ,   5.7650491,  10.0

### 2.1 Generation of all possible Monomials

In [18]:
polys_symbols = symbols(list(variables))
polys_monomials = sorted(monomials.itermonomials(variables = polys_symbols, max_degrees = d, min_degrees = None), 
                                      reverse = True, key = monomial_key('lex', polys_symbols))
monomials_count = monomials.monomial_count(n, d)

tuples_monomials = [ph.monomial_to_power_tuple(mon, polys_symbols) for mon in polys_monomials]

print('List length: ' + str(len(polys_monomials)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(monomials_count))
print(polys_monomials)
print(tuples_monomials)

List length: 35
Number of monomials in a polynomial with 4 variables and degree 3: 35
[a**3, a**2*b, a**2*c, a**2*d, a**2, a*b**2, a*b*c, a*b*d, a*b, a*c**2, a*c*d, a*c, a*d**2, a*d, a, b**3, b**2*c, b**2*d, b**2, b*c**2, b*c*d, b*c, b*d**2, b*d, b, c**3, c**2*d, c**2, c*d**2, c*d, c, d**3, d**2, d, 1]
[(3, 0, 0, 0), (2, 1, 0, 0), (2, 0, 1, 0), (2, 0, 0, 1), (2, 0, 0, 0), (1, 2, 0, 0), (1, 1, 1, 0), (1, 1, 0, 1), (1, 1, 0, 0), (1, 0, 2, 0), (1, 0, 1, 1), (1, 0, 1, 0), (1, 0, 0, 2), (1, 0, 0, 1), (1, 0, 0, 0), (0, 3, 0, 0), (0, 2, 1, 0), (0, 2, 0, 1), (0, 2, 0, 0), (0, 1, 2, 0), (0, 1, 1, 1), (0, 1, 1, 0), (0, 1, 0, 2), (0, 1, 0, 1), (0, 1, 0, 0), (0, 0, 3, 0), (0, 0, 2, 1), (0, 0, 2, 0), (0, 0, 1, 2), (0, 0, 1, 1), (0, 0, 1, 0), (0, 0, 0, 3), (0, 0, 0, 2), (0, 0, 0, 1), (0, 0, 0, 0)]


In [19]:
#clear files
if each_epochs_save != None:
    try:
        # Create target Directory
        os.mkdir('./data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename)
    except FileExistsError:

        for i in range(epochs//each_epochs_save):    
            path = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size)  + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str((i+1)*each_epochs_save).zfill(3)  + filename + '.txt'

            with open(path, 'wt') as text_file:
                text_file.truncate()   
    try:
        # Create target Directory
        os.mkdir('./data/results/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename)
    except FileExistsError:
        pass
else:
    
    try:
        # Create target Directory
        os.mkdir('./data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename)
    except FileExistsError:

        path = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'

        with open(path, 'wt') as text_file:
            text_file.truncate()   
    try:
        # Create target Directory
        os.mkdir('./data/results/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename)
    except FileExistsError:
        pass
                

## Lambda Network Training + Weigh/Bias saving

In [20]:
def train_nn(X_data, y_data, polynomial, callbacks=None, return_history=False, each_epochs_save=None, printing=False, return_model=False):
    
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    if int(tf.__version__[0]) >= 2:
        tf.random.set_seed(RANDOM_SEED)
    else:
        tf.set_random_seed(RANDOM_SEED) 
        
    if isinstance(X_data, pd.DataFrame):
        X_data = X_data.values
    if isinstance(y_data, pd.DataFrame):
        y_data = y_data.values
        
    X_train_with_valid, X_test, y_train_with_valid, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=RANDOM_SEED)           
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_with_valid, y_train_with_valid, test_size=0.25, random_state=RANDOM_SEED)           
     
    model = Sequential()

    model.add(Dense(lambda_network_layers[0], activation='relu', input_dim=X_data.shape[1])) #1024
    
    if dropout > 0:
        model.add(Dropout(dropout))

    for neurons in lambda_network_layers[1:]:
        model.add(Dense(neurons, activation='relu'))
        if dropout > 0:
            model.add(Dropout(dropout))   
            
    model.add(Dense(1))
    
    model.compile(optimizer=optimizer,
                  loss='mae', #huber_loss(val_min, val_max), #'mape',#'mean_absolute_error',#root_mean_squared_error,
                  metrics=[mean_absolute_percentage_error_keras, root_mean_squared_error]
                 )
    
    weights = []
    polynomial_pred_list = []
    polynomial_lstsq_true_list = []
        
        

     
    if each_epochs_save == None:        
        model_history = model.fit(X_train, 
                      y_train, 
                      epochs=epochs, 
                      batch_size=batch_size, 
                      callbacks=callbacks,
                      validation_data=(X_valid, y_valid),
                      verbose=0,
                      workers=0)
        weights.append(model.get_weights())
        
        history = model_history.history
        

        y_pred_valid = model.predict(X_valid)                
        y_pred_test = model.predict(X_test)

        term_list_all = []
        y = 0
        for constant, term in zip(polynomial.values, list(polynomial.index)):
            term_list = [int(value_mult) for value_mult in term]
            term_list_all.append(term_list)

        #print(trm_list_all)

        #generate separate arrays for each variable combination
        terms_matrix = []
        for unknowns in X_valid:
            terms = []
            for term_multipliers in term_list_all:
                term_value = prod([unknown**multiplier for unknown, multiplier in zip(unknowns, term_multipliers)])

                terms.append(term_value)
            terms_matrix.append(np.array(terms))

        terms_matrix = np.array(terms_matrix)

        polynomial_pred, _, _, _ = np.linalg.lstsq(terms_matrix, y_pred_valid.ravel(), rcond=-1)#[::-1]
        polynomial_lstsq_true, _, _, _ = np.linalg.lstsq(terms_matrix, y_valid.ravel(), rcond=-1)#[::-1] 
        polynomial_pred_list.append(polynomial_pred)
        polynomial_lstsq_true_list.append(polynomial_lstsq_true)
        
        y_test_polynomial = []
        y_test_lstsq = []
        for entry in X_test:
            true_function_value_pred, _ = calcualate_function_with_data(polynomial_pred, entry)
            true_function_value_lstsq, _ = calcualate_function_with_data(polynomial_lstsq_true, entry)
            
            y_test_polynomial.append(true_function_value_pred)
            y_test_lstsq.append(true_function_value_lstsq)
        y_test_polynomial = np.array(y_test_polynomial).reshape(len(y_test_polynomial), 1)
        y_test_lstsq = np.array(y_test_lstsq).reshape(len(y_test_lstsq), 1)


        y_valid_polynomial = []  
        y_valid_lstsq = []
        for entry in X_valid:
            true_function_value_pred, _ = calcualate_function_with_data(polynomial_pred, entry)
            true_function_value_lstsq, _ = calcualate_function_with_data(polynomial_lstsq_true, entry)
            y_valid_polynomial.append(true_function_value_pred)     
            y_valid_lstsq.append(true_function_value_lstsq)     
        y_valid_polynomial = np.array(y_valid_polynomial).reshape(len(y_valid_polynomial), 1)     
        y_valid_lstsq = np.array(y_valid_lstsq).reshape(len(y_valid_lstsq), 1)     

        pred_list = (y_valid, y_pred_valid, y_valid_polynomial, X_valid, y_test, y_pred_test, y_test_polynomial, X_test)

        mae_test_pred = np.round(mean_absolute_error(y_test, y_pred_test), 4)
        mae_test_polynomial = np.round(mean_absolute_error(y_test, y_test_polynomial), 4)
        mae_test_polynomial_pred = np.round(mean_absolute_error(y_test_polynomial, y_pred_test), 4)
        mae_test_lstsq = np.round(mean_absolute_error(y_test, y_test_lstsq), 4)    

        rmse_test_pred = np.round(root_mean_squared_error(y_test, y_pred_test), 4)    
        rmse_test_polynomial = np.round(root_mean_squared_error(y_test, y_test_polynomial), 4)    
        rmse_test_polynomial_pred = np.round(root_mean_squared_error(y_test_polynomial, y_pred_test), 4)    
        rmse_test_lstsq = np.round(root_mean_squared_error(y_test, y_test_lstsq), 4)    
        
        mape_test_pred = np.round(mean_absolute_percentage_error_keras(y_test, y_pred_test), 4)    
        mape_test_polynomial = np.round(mean_absolute_percentage_error_keras(y_test, y_test_polynomial), 4)    
        mape_test_polynomial_pred = np.round(mean_absolute_percentage_error_keras(y_test_polynomial, y_pred_test), 4)    
        mape_test_lstsq = np.round(mean_absolute_percentage_error_keras(y_test, y_test_lstsq), 4)            

        r2_test_pred = np.round(r2_score(y_test, y_pred_test), 4)
        r2_test_polynomial = np.round(r2_score(y_test, y_test_polynomial), 4)
        r2_test_polynomial_pred = np.round(r2_score(y_test_polynomial, y_pred_test), 4)
        r2_test_lstsq = np.round(r2_score(y_test, y_test_lstsq), 4)
        
        raae_test_pred = np.round(relative_absolute_average_error(y_test, y_pred_test), 4)
        raae_test_polynomial = np.round(relative_absolute_average_error(y_test, y_test_polynomial), 4)
        raae_test_polynomial_pred = np.round(relative_absolute_average_error(y_test_polynomial, y_pred_test), 4)
        raae_test_lstsq = np.round(relative_absolute_average_error(y_test, y_test_lstsq), 4)
        
        rmae_test_pred = np.round(relative_maximum_average_error(y_test, y_pred_test), 4)
        rmae_test_polynomial = np.round(relative_maximum_average_error(y_test, y_test_polynomial), 4)
        rmae_test_polynomial_pred = np.round(relative_maximum_average_error(y_test_polynomial, y_pred_test), 4)
        rmae_test_lstsq = np.round(relative_maximum_average_error(y_test, y_test_lstsq), 4)
        
        fd_test_pred = np.round(frechet_dist(np.column_stack((X_test[:advanced_metric_dataset_size], y_test[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_pred_test[:advanced_metric_dataset_size]))), 4)
        fd_test_polynomial = np.round(frechet_dist(np.column_stack((X_test[:advanced_metric_dataset_size], y_test[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_test_polynomial[:advanced_metric_dataset_size]))), 4)
        fd_test_polynomial_pred = np.round(frechet_dist(np.column_stack((X_test[:advanced_metric_dataset_size], y_test_polynomial[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_pred_test[:advanced_metric_dataset_size]))), 4)
        fd_test_lstsq = np.round(frechet_dist(np.column_stack((X_test[:advanced_metric_dataset_size], y_test[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_test_lstsq[:advanced_metric_dataset_size]))), 4)
        
        dtw_test_pred, dtw_complete_test_pred = dtw(np.column_stack((X_test[:advanced_metric_dataset_size], y_test[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_pred_test[:advanced_metric_dataset_size])))
        dtw_test_pred = np.round(dtw_test_pred, 4)
        dtw_test_polynomial, dtw_complete_test_polynomial = dtw(np.column_stack((X_test[:advanced_metric_dataset_size], y_test[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_test_polynomial[:advanced_metric_dataset_size])))
        dtw_test_polynomial = np.round(dtw_test_polynomial, 4)
        dtw_test_polynomial_pred, dtw_complete_test_polynomial_pred = dtw(np.column_stack((X_test[:advanced_metric_dataset_size], y_test_polynomial[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_pred_test[:advanced_metric_dataset_size])))
        dtw_test_polynomial_pred = np.round(dtw_test_polynomial_pred, 4)    
        dtw_test_lstsq, dtw_complete_test_lstsq = dtw(np.column_stack((X_test[:advanced_metric_dataset_size], y_test[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_test_lstsq[:advanced_metric_dataset_size])))
        dtw_test_lstsq = np.round(dtw_test_lstsq, 4)        


        mae_valid_pred = np.round(mean_absolute_error(y_valid, y_pred_valid), 4)
        mae_valid_polynomial = np.round(mean_absolute_error(y_valid, y_valid_polynomial), 4)
        mae_valid_polynomial_pred = np.round(mean_absolute_error(y_valid_polynomial, y_pred_valid), 4)
        mae_valid_lstsq = np.round(mean_absolute_error(y_valid, y_valid_lstsq), 4)
        
        rmse_valid_pred = np.round(root_mean_squared_error(y_valid, y_pred_valid), 4)
        rmse_valid_polynomial = np.round(root_mean_squared_error(y_valid, y_valid_polynomial), 4)
        rmse_valid_polynomial_pred = np.round(root_mean_squared_error(y_valid_polynomial, y_pred_valid), 4)
        rmse_valid_lstsq = np.round(root_mean_squared_error(y_valid, y_valid_lstsq), 4)
        
        mape_valid_pred = np.round(mean_absolute_percentage_error_keras(y_valid, y_pred_valid), 4)
        mape_valid_polynomial = np.round(mean_absolute_percentage_error_keras(y_valid, y_valid_polynomial), 4)
        mape_valid_polynomial_pred = np.round(mean_absolute_percentage_error_keras(y_valid_polynomial, y_pred_valid), 4)
        mape_valid_lstsq = np.round(mean_absolute_percentage_error_keras(y_valid, y_valid_lstsq), 4)
        
        r2_valid_pred = np.round(r2_score(y_valid, y_pred_valid), 4)
        r2_valid_polynomial = np.round(r2_score(y_valid, y_valid_polynomial), 4)
        r2_valid_polynomial_pred = np.round(r2_score(y_valid_polynomial, y_pred_valid), 4)
        r2_valid_lstsq = np.round(r2_score(y_valid, y_valid_lstsq), 4)
        
        raae_valid_pred = np.round(relative_absolute_average_error(y_valid, y_pred_valid), 4)
        raae_valid_polynomial = np.round(relative_absolute_average_error(y_valid, y_valid_polynomial), 4)
        raae_valid_polynomial_pred = np.round(relative_absolute_average_error(y_valid_polynomial, y_pred_valid), 4)
        raae_valid_lstsq = np.round(relative_absolute_average_error(y_valid, y_valid_lstsq), 4)
        
        rmae_valid_pred = np.round(relative_maximum_average_error(y_valid, y_pred_valid), 4) 
        rmae_valid_polynomial = np.round(relative_maximum_average_error(y_valid, y_valid_polynomial), 4) 
        rmae_valid_polynomial_pred = np.round(relative_maximum_average_error(y_valid_polynomial, y_pred_valid), 4) 
        rmae_valid_lstsq = np.round(relative_maximum_average_error(y_valid, y_valid_lstsq), 4) 
        
        fd_valid_pred = np.round(frechet_dist(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_pred_valid[:advanced_metric_dataset_size]))), 4)
        fd_valid_polynomial = np.round(frechet_dist(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid_polynomial[:advanced_metric_dataset_size]))), 4)
        fd_valid_polynomial_pred = np.round(frechet_dist(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid_polynomial[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_pred_valid[:advanced_metric_dataset_size]))), 4)
        fd_valid_lstsq = np.round(frechet_dist(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid_lstsq[:advanced_metric_dataset_size]))), 4)
        
        dtw_valid_pred, dtw_complete_valid_pred = dtw(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_pred_valid[:advanced_metric_dataset_size])))
        dtw_valid_pred = np.round(dtw_valid_pred, 4)    
        dtw_valid_polynomial, dtw_complete_valid_polynomial = dtw(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid_polynomial[:advanced_metric_dataset_size])))
        dtw_valid_polynomial = np.round(dtw_valid_polynomial, 4)       
        dtw_valid_polynomial_pred, dtw_complete_valid_polynomial_pred = dtw(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid_polynomial[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_pred_valid[:advanced_metric_dataset_size])))
        dtw_valid_polynomial_pred = np.round(dtw_valid_polynomial_pred, 4)   
        dtw_valid_lstsq, dtw_complete_valid_lstsq = dtw(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid_lstsq[:advanced_metric_dataset_size])))
        dtw_valid_lstsq = np.round(dtw_valid_lstsq, 4)

        
        std_train_real = np.std(y_train)
        std_valid_real = np.std(y_valid)
        std_test_real = np.std(y_test)

        std_valid_pred = np.std(y_pred_valid)
        std_test_pred = np.std(y_pred_test)

        std_valid_polynomial = np.std(y_valid_polynomial)
        std_test_polynomial = np.std(y_test_polynomial)

        std_valid_lstsq = np.std(y_valid_lstsq)
        std_test_lstsq = np.std(y_test_lstsq)
        
        
        mean_train_real = np.mean(y_train)
        mean_valid_real = np.mean(y_valid)
        mean_test_real = np.mean(y_test)

        mean_valid_pred = np.mean(y_pred_valid)
        mean_test_pred = np.mean(y_pred_test)

        mean_valid_polynomial = np.mean(y_valid_polynomial)
        mean_test_polynomial = np.mean(y_test_polynomial)

        mean_valid_lstsq = np.mean(y_valid_lstsq)
        mean_test_lstsq = np.mean(y_test_lstsq)

        result_dict_list =  [{
                         'MAE FV VALID PRED': mae_valid_pred,
                         'MAE FV VALID POLY': mae_valid_polynomial,
                         'MAE FV VALID POLY PRED': mae_valid_polynomial_pred,
                         'MAE FV VALID LSTSQ': mae_valid_lstsq,
                         'RMSE FV VALID PRED': rmse_valid_pred,
                         'RMSE FV VALID POLY': rmse_valid_polynomial,
                         'RMSE FV VALID POLY PRED': rmse_valid_polynomial_pred,
                         'RMSE FV VALID LSTSQ': rmse_valid_lstsq,
                         'MAPE FV VALID PRED': mape_valid_pred,
                         'MAPE FV VALID POLY': mape_valid_polynomial,
                         'MAPE FV VALID POLY PRED': mape_valid_polynomial_pred,
                         'MAPE FV VALID LSTSQ': mape_valid_lstsq,
                         'R2 FV VALID PRED': r2_valid_pred,
                         'R2 FV VALID POLY': r2_valid_polynomial,
                         'R2 FV VALID POLY PRED': r2_valid_polynomial_pred,
                         'R2 FV VALID LSTSQ': r2_valid_lstsq,
                         'RAAE FV PRED VALID': raae_valid_pred,
                         'RAAE FV VALID POLY': raae_valid_polynomial,
                         'RAAE FV VALID POLY PRED': raae_valid_polynomial_pred,
                         'RAAE FV VALID LSTSQ': raae_valid_lstsq,
                         'RMAE FV VALID PRED': rmae_valid_pred,
                         'RMAE FV VALID POLY': rmae_valid_polynomial,
                         'RMAE FV VALID POLY PRED': rmae_valid_polynomial_pred,
                         'RMAE FV VALID LSTSQ': rmae_valid_lstsq,
                         'FD FV VALID PRED': fd_valid_pred,   
                         'FD FV VALID POLY': fd_valid_polynomial,   
                         'FD FV VALID POLY PRED': fd_valid_polynomial_pred,   
                         'FD FV VALID LSTSQ': fd_valid_lstsq,   
                         'DTW FV VALID PRED': dtw_valid_pred, 
                         'DTW FV VALID POLY': dtw_valid_polynomial, 
                         'DTW FV VALID POLY PRED': dtw_valid_polynomial_pred, 
                         'DTW FV VALID LSTSQ': dtw_valid_lstsq, 
        },{
                         'MAE FV TEST PRED': mae_test_pred,
                         'MAE FV TEST POLY': mae_test_polynomial,
                         'MAE FV TEST POLY PRED': mae_test_polynomial_pred,
                         'MAE FV TEST LSTSQ': mae_test_lstsq,
                         'RMSE FV TEST PRED': rmse_test_pred,
                         'RMSE FV TEST POLY': rmse_test_polynomial,
                         'RMSE FV TEST POLY PRED': rmse_test_polynomial_pred,
                         'RMSE FV TEST LSTSQ': rmse_test_lstsq,
                         'MAPE FV TEST PRED': mape_test_pred,
                         'MAPE FV TEST POLY': mape_test_polynomial,
                         'MAPE FV TEST POLY PRED': mape_test_polynomial_pred,
                         'MAPE FV TEST LSTSQ': mape_test_lstsq,
                         'R2 FV TEST PRED': r2_test_pred,
                         'R2 FV TEST POLY': r2_test_polynomial,
                         'R2 FV TEST POLY PRED': r2_test_polynomial_pred,
                         'R2 FV TEST LSTSQ': r2_test_lstsq,
                         'RAAE FV TEST PRED': raae_test_pred,
                         'RAAE FV TEST POLY': raae_test_polynomial,
                         'RAAE FV TEST POLY PRED': raae_test_polynomial_pred,
                         'RAAE FV TEST LSTSQ': raae_test_lstsq,
                         'RMAE FV TEST PRED': rmae_test_pred,
                         'RMAE FV TEST POLY': rmae_test_polynomial,
                         'RMAE FV TEST POLY PRED': rmae_test_polynomial_pred,        
                         'RMAE FV TEST LSTSQ': rmae_test_lstsq,
                         'FD FV TEST PRED': fd_test_pred,    
                         'FD FV TEST POLY': fd_test_polynomial,    
                         'FD FV TEST POLY PRED': fd_test_polynomial_pred, 
                         'FD FV TEST LSTSQ': fd_test_lstsq,    
                         'DTW FV TEST PRED': dtw_test_pred,
                         'DTW FV TEST POLY': dtw_test_polynomial,
                         'DTW FV TEST POLY PRED': dtw_test_polynomial_pred, 
                         'DTW FV TEST LSTSQ': dtw_test_lstsq,
        },{
                         'STD FV TRAIN REAL': std_train_real, 
                         'STD FV VALID REAL': std_valid_real, 
                         'STD FV VALID PRED': std_valid_pred,
                         'STD FV VALID POLY': std_valid_polynomial, 
                         'STD FV VALID LSTSQ': std_valid_lstsq, 
                         'STD FV TEST REAL': std_test_real,
                         'STD FV TEST PRED': std_test_pred, 
                         'STD FV TEST POLY': std_test_polynomial, 
                         'STD FV TEST LSTSQ': std_test_lstsq, 
        },{
                         'MEAN FV TRAIN REAL': mean_train_real, 
                         'MEAN FV VALID REAL': mean_valid_real, 
                         'MEAN FV VALID PRED': mean_valid_pred,
                         'MEAN FV VALID POLY': mean_valid_polynomial, 
                         'MEAN FV VALID LSTSQ': mean_valid_lstsq, 
                         'MEAN FV TEST REAL': mean_test_real,
                         'MEAN FV TEST PRED': mean_test_pred, 
                         'MEAN FV TEST POLY': mean_test_polynomial, 
                         'MEAN FV TEST LSTSQ': mean_test_lstsq, 
        }]            
                            
    else:
        result_dict_list = []
        pred_list = []
        for i in range(epochs//each_epochs_save):
            if i == 0:
                history = model.fit(X_train, 
                          y_train, 
                          epochs=each_epochs_save, 
                          batch_size=batch_size, 
                          callbacks=callbacks,
                          validation_data=(X_valid, y_valid),
                          verbose=0,
                          workers=1,
                          use_multiprocessing=False
                          )
                history = history.history
            else:
                model_history = model.fit(X_train, 
                          y_train, 
                          epochs=each_epochs_save, 
                          batch_size=batch_size, 
                          callbacks=callbacks,
                          validation_data=(X_valid, y_valid),
                          verbose=0,
                          workers=1,
                          use_multiprocessing=False
                          )

                for key_1 in history.keys():
                    for key_2 in model_history.history.keys():
                        if key_1 == key_2:
                            history[key_1] += model_history.history[key_2]  
            
            weights.append(model.get_weights())
            

            y_pred_valid = model.predict(X_valid)                
            y_pred_test = model.predict(X_test)        

            term_list_all = []
            y = 0
            for constant, term in zip(polynomial.values, list(polynomial.index)):
                term_list = [int(value_mult) for value_mult in term]
                term_list_all.append(term_list)


            terms_matrix = []
            for unknowns in X_valid:
                terms = []
                for term_multipliers in term_list_all:
                    term_value = prod([unknown**multiplier for unknown, multiplier in zip(unknowns, term_multipliers)])

                    terms.append(term_value)
                terms_matrix.append(np.array(terms))

            terms_matrix = np.array(terms_matrix)

            polynomial_pred, _, _, _ = np.linalg.lstsq(terms_matrix, y_pred_valid.ravel(), rcond=-1)#[::-1] 
            #does not change over time
            if i == 0:
                polynomial_lstsq_true, _, _, _ = np.linalg.lstsq(terms_matrix, y_valid.ravel(), rcond=-1)#[::-1] 
            polynomial_pred_list.append(polynomial_pred)
            polynomial_lstsq_true_list.append(polynomial_lstsq_true)            

            y_test_polynomial = []
            if i == 0:
                y_test_lstsq = []
            for entry in X_test:
                true_function_value_pred, _ = calcualate_function_with_data(polynomial_pred, entry)
                y_test_polynomial.append(true_function_value_pred)
                if i == 0:
                    true_function_value_lstsq, _ = calcualate_function_with_data(polynomial_lstsq_true, entry)
                    y_test_lstsq.append(true_function_value_lstsq)

            y_test_polynomial = np.array(y_test_polynomial).reshape(len(y_test_polynomial), 1)
            if i == 0:
                y_test_lstsq = np.array(y_test_lstsq).reshape(len(y_test_lstsq), 1)


            y_valid_polynomial = []  
            if i == 0:
                y_valid_lstsq = []
            for entry in X_valid:
                true_function_value_pred, _ = calcualate_function_with_data(polynomial_pred, entry)
                y_valid_polynomial.append(true_function_value_pred)   
                if i == 0:
                    true_function_value_lstsq, _ = calcualate_function_with_data(polynomial_lstsq_true, entry)
                    y_valid_lstsq.append(true_function_value_lstsq)     
            y_valid_polynomial = np.array(y_valid_polynomial).reshape(len(y_valid_polynomial), 1)     
            if i == 0:
                y_valid_lstsq = np.array(y_valid_lstsq).reshape(len(y_valid_lstsq), 1)    
                
            pred_list.append((y_valid, y_pred_valid, y_valid_polynomial, X_valid, y_test, y_pred_test, y_test_polynomial, X_test))

            
            mae_test_pred = np.round(mean_absolute_error(y_test, y_pred_test), 4)
            mae_test_polynomial = np.round(mean_absolute_error(y_test, y_test_polynomial), 4)
            mae_test_polynomial_pred = np.round(mean_absolute_error(y_test_polynomial, y_pred_test), 4)
            if i == 0:
                mae_test_lstsq = np.round(mean_absolute_error(y_test, y_test_lstsq), 4)    

            rmse_test_pred = np.round(root_mean_squared_error(y_test, y_pred_test), 4)    
            rmse_test_polynomial = np.round(root_mean_squared_error(y_test, y_test_polynomial), 4)    
            rmse_test_polynomial_pred = np.round(root_mean_squared_error(y_test_polynomial, y_pred_test), 4)    
            if i == 0:
                rmse_test_lstsq = np.round(root_mean_squared_error(y_test, y_test_lstsq), 4)    

            mape_test_pred = np.round(mean_absolute_percentage_error_keras(y_test, y_pred_test), 4)    
            mape_test_polynomial = np.round(mean_absolute_percentage_error_keras(y_test, y_test_polynomial), 4)    
            mape_test_polynomial_pred = np.round(mean_absolute_percentage_error_keras(y_test_polynomial, y_pred_test), 4)    
            if i == 0:
                mape_test_lstsq = np.round(mean_absolute_percentage_error_keras(y_test, y_test_lstsq), 4)            

            r2_test_pred = np.round(r2_score(y_test, y_pred_test), 4)
            r2_test_polynomial = np.round(r2_score(y_test, y_test_polynomial), 4)
            r2_test_polynomial_pred = np.round(r2_score(y_test_polynomial, y_pred_test), 4)
            if i == 0:
                r2_test_lstsq = np.round(r2_score(y_test, y_test_lstsq), 4)

            raae_test_pred = np.round(relative_absolute_average_error(y_test, y_pred_test), 4)
            raae_test_polynomial = np.round(relative_absolute_average_error(y_test, y_test_polynomial), 4)
            raae_test_polynomial_pred = np.round(relative_absolute_average_error(y_test_polynomial, y_pred_test), 4)
            if i == 0:
                raae_test_lstsq = np.round(relative_absolute_average_error(y_test, y_test_lstsq), 4)

            rmae_test_pred = np.round(relative_maximum_average_error(y_test, y_pred_test), 4)
            rmae_test_polynomial = np.round(relative_maximum_average_error(y_test, y_test_polynomial), 4)
            rmae_test_polynomial_pred = np.round(relative_maximum_average_error(y_test_polynomial, y_pred_test), 4)
            if i == 0:
                rmae_test_lstsq = np.round(relative_maximum_average_error(y_test, y_test_lstsq), 4)

            fd_test_pred = np.round(frechet_dist(np.column_stack((X_test[:advanced_metric_dataset_size], y_test[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_pred_test[:advanced_metric_dataset_size]))), 4)
            fd_test_polynomial = np.round(frechet_dist(np.column_stack((X_test[:advanced_metric_dataset_size], y_test[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_test_polynomial[:advanced_metric_dataset_size]))), 4)
            fd_test_polynomial_pred = np.round(frechet_dist(np.column_stack((X_test[:advanced_metric_dataset_size], y_test_polynomial[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_pred_test[:advanced_metric_dataset_size]))), 4)
            if i == 0:
                fd_test_lstsq = np.round(frechet_dist(np.column_stack((X_test[:advanced_metric_dataset_size], y_test[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_test_lstsq[:advanced_metric_dataset_size]))), 4)

            dtw_test_pred, dtw_complete_test_pred = dtw(np.column_stack((X_test[:advanced_metric_dataset_size], y_test[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_pred_test[:advanced_metric_dataset_size])))
            dtw_test_pred = np.round(dtw_test_pred, 4)
            dtw_test_polynomial, dtw_complete_test_polynomial = dtw(np.column_stack((X_test[:advanced_metric_dataset_size], y_test[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_test_polynomial[:advanced_metric_dataset_size])))
            dtw_test_polynomial = np.round(dtw_test_polynomial, 4)
            dtw_test_polynomial_pred, dtw_complete_test_polynomial_pred = dtw(np.column_stack((X_test[:advanced_metric_dataset_size], y_test_polynomial[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_pred_test[:advanced_metric_dataset_size])))
            dtw_test_polynomial_pred = np.round(dtw_test_polynomial_pred, 4)    
            if i == 0:
                dtw_test_lstsq, dtw_complete_test_lstsq = dtw(np.column_stack((X_test[:advanced_metric_dataset_size], y_test[:advanced_metric_dataset_size])), np.column_stack((X_test[:advanced_metric_dataset_size], y_test_lstsq[:advanced_metric_dataset_size])))
                dtw_test_lstsq = np.round(dtw_test_lstsq, 4)        


            mae_valid_pred = np.round(mean_absolute_error(y_valid, y_pred_valid), 4)
            mae_valid_polynomial = np.round(mean_absolute_error(y_valid, y_valid_polynomial), 4)
            mae_valid_polynomial_pred = np.round(mean_absolute_error(y_valid_polynomial, y_pred_valid), 4)
            if i == 0:
                mae_valid_lstsq = np.round(mean_absolute_error(y_valid, y_valid_lstsq), 4)

            rmse_valid_pred = np.round(root_mean_squared_error(y_valid, y_pred_valid), 4)
            rmse_valid_polynomial = np.round(root_mean_squared_error(y_valid, y_valid_polynomial), 4)
            rmse_valid_polynomial_pred = np.round(root_mean_squared_error(y_valid_polynomial, y_pred_valid), 4)
            if i == 0:
                rmse_valid_lstsq = np.round(root_mean_squared_error(y_valid, y_valid_lstsq), 4)

            mape_valid_pred = np.round(mean_absolute_percentage_error_keras(y_valid, y_pred_valid), 4)
            mape_valid_polynomial = np.round(mean_absolute_percentage_error_keras(y_valid, y_valid_polynomial), 4)
            mape_valid_polynomial_pred = np.round(mean_absolute_percentage_error_keras(y_valid_polynomial, y_pred_valid), 4)
            if i == 0:
                mape_valid_lstsq = np.round(mean_absolute_percentage_error_keras(y_valid, y_valid_lstsq), 4)

            r2_valid_pred = np.round(r2_score(y_valid, y_pred_valid), 4)
            r2_valid_polynomial = np.round(r2_score(y_valid, y_valid_polynomial), 4)
            r2_valid_polynomial_pred = np.round(r2_score(y_valid_polynomial, y_pred_valid), 4)
            if i == 0:
                r2_valid_lstsq = np.round(r2_score(y_valid, y_valid_lstsq), 4)

            raae_valid_pred = np.round(relative_absolute_average_error(y_valid, y_pred_valid), 4)
            raae_valid_polynomial = np.round(relative_absolute_average_error(y_valid, y_valid_polynomial), 4)
            raae_valid_polynomial_pred = np.round(relative_absolute_average_error(y_valid_polynomial, y_pred_valid), 4)
            if i == 0:
                raae_valid_lstsq = np.round(relative_absolute_average_error(y_valid, y_valid_lstsq), 4)

            rmae_valid_pred = np.round(relative_maximum_average_error(y_valid, y_pred_valid), 4) 
            rmae_valid_polynomial = np.round(relative_maximum_average_error(y_valid, y_valid_polynomial), 4) 
            rmae_valid_polynomial_pred = np.round(relative_maximum_average_error(y_valid_polynomial, y_pred_valid), 4) 
            if i == 0:
                rmae_valid_lstsq = np.round(relative_maximum_average_error(y_valid, y_valid_lstsq), 4) 

            fd_valid_pred = np.round(frechet_dist(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_pred_valid[:advanced_metric_dataset_size]))), 4)
            fd_valid_polynomial = np.round(frechet_dist(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid_polynomial[:advanced_metric_dataset_size]))), 4)
            fd_valid_polynomial_pred = np.round(frechet_dist(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid_polynomial[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_pred_valid[:advanced_metric_dataset_size]))), 4)
            if i == 0:
                fd_valid_lstsq = np.round(frechet_dist(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid_lstsq[:advanced_metric_dataset_size]))), 4)

            dtw_valid_pred, dtw_complete_valid_pred = dtw(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_pred_valid[:advanced_metric_dataset_size])))
            dtw_valid_pred = np.round(dtw_valid_pred, 4)    
            dtw_valid_polynomial, dtw_complete_valid_polynomial = dtw(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid_polynomial[:advanced_metric_dataset_size])))
            dtw_valid_polynomial = np.round(dtw_valid_polynomial, 4)       
            dtw_valid_polynomial_pred, dtw_complete_valid_polynomial_pred = dtw(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid_polynomial[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_pred_valid[:advanced_metric_dataset_size])))
            dtw_valid_polynomial_pred = np.round(dtw_valid_polynomial_pred, 4)   
            if i == 0:
                dtw_valid_lstsq, dtw_complete_valid_lstsq = dtw(np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid[:advanced_metric_dataset_size])), np.column_stack((X_valid[:advanced_metric_dataset_size], y_valid_lstsq[:advanced_metric_dataset_size])))
                dtw_valid_lstsq = np.round(dtw_valid_lstsq, 4)

            std_train_real = np.std(y_train)
            std_valid_real = np.std(y_valid)
            std_test_real = np.std(y_test)

            std_valid_pred = np.std(y_pred_valid)
            std_test_pred = np.std(y_pred_test)

            std_valid_polynomial = np.std(y_valid_polynomial)
            std_test_polynomial = np.std(y_test_polynomial)

            if i == 0:
                std_valid_lstsq = np.std(y_valid_lstsq)
                std_test_lstsq = np.std(y_test_lstsq)
                
                
            mean_train_real = np.mean(y_train)
            mean_valid_real = np.mean(y_valid)
            mean_test_real = np.mean(y_test)

            mean_valid_pred = np.mean(y_pred_valid)
            mean_test_pred = np.mean(y_pred_test)

            mean_valid_polynomial = np.mean(y_valid_polynomial)
            mean_test_polynomial = np.mean(y_test_polynomial)

            if i == 0:
                mean_valid_lstsq = np.mean(y_valid_lstsq)
                mean_test_lstsq = np.mean(y_test_lstsq)
                
                
            result_dict_list_single_epoch =  [{
                         'MAE FV VALID PRED': mae_valid_pred,
                         'MAE FV VALID POLY': mae_valid_polynomial,
                         'MAE FV VALID POLY PRED': mae_valid_polynomial_pred,
                         'MAE FV VALID LSTSQ': mae_valid_lstsq,
                         'RMSE FV VALID PRED': rmse_valid_pred,
                         'RMSE FV VALID POLY': rmse_valid_polynomial,
                         'RMSE FV VALID POLY PRED': rmse_valid_polynomial_pred,
                         'RMSE FV VALID LSTSQ': rmse_valid_lstsq,
                         'MAPE FV VALID PRED': mape_valid_pred,
                         'MAPE FV VALID POLY': mape_valid_polynomial,
                         'MAPE FV VALID POLY PRED': mape_valid_polynomial_pred,
                         'MAPE FV VALID LSTSQ': mape_valid_lstsq,
                         'R2 FV VALID PRED': r2_valid_pred,
                         'R2 FV VALID POLY': r2_valid_polynomial,
                         'R2 FV VALID POLY PRED': r2_valid_polynomial_pred,
                         'R2 FV VALID LSTSQ': r2_valid_lstsq,
                         'RAAE FV PRED VALID': raae_valid_pred,
                         'RAAE FV VALID POLY': raae_valid_polynomial,
                         'RAAE FV VALID POLY PRED': raae_valid_polynomial_pred,
                         'RAAE FV VALID LSTSQ': raae_valid_lstsq,
                         'RMAE FV VALID PRED': rmae_valid_pred,
                         'RMAE FV VALID POLY': rmae_valid_polynomial,
                         'RMAE FV VALID POLY PRED': rmae_valid_polynomial_pred,
                         'RMAE FV VALID LSTSQ': rmae_valid_lstsq,
                         'FD FV VALID PRED': fd_valid_pred,   
                         'FD FV VALID POLY': fd_valid_polynomial,   
                         'FD FV VALID POLY PRED': fd_valid_polynomial_pred,   
                         'FD FV VALID LSTSQ': fd_valid_lstsq,   
                         'DTW FV VALID PRED': dtw_valid_pred, 
                         'DTW FV VALID POLY': dtw_valid_polynomial, 
                         'DTW FV VALID POLY PRED': dtw_valid_polynomial_pred, 
                         'DTW FV VALID LSTSQ': dtw_valid_lstsq, 
        },{
                         'MAE FV TEST PRED': mae_test_pred,
                         'MAE FV TEST POLY': mae_test_polynomial,
                         'MAE FV TEST POLY PRED': mae_test_polynomial_pred,
                         'MAE FV TEST LSTSQ': mae_test_lstsq,
                         'RMSE FV TEST PRED': rmse_test_pred,
                         'RMSE FV TEST POLY': rmse_test_polynomial,
                         'RMSE FV TEST POLY PRED': rmse_test_polynomial_pred,
                         'RMSE FV TEST LSTSQ': rmse_test_lstsq,
                         'MAPE FV TEST PRED': mape_test_pred,
                         'MAPE FV TEST POLY': mape_test_polynomial,
                         'MAPE FV TEST POLY PRED': mape_test_polynomial_pred,
                         'MAPE FV TEST LSTSQ': mape_test_lstsq,
                         'R2 FV TEST PRED': r2_test_pred,
                         'R2 FV TEST POLY': r2_test_polynomial,
                         'R2 FV TEST POLY PRED': r2_test_polynomial_pred,
                         'R2 FV TEST LSTSQ': r2_test_lstsq,
                         'RAAE FV TEST PRED': raae_test_pred,
                         'RAAE FV TEST POLY': raae_test_polynomial,
                         'RAAE FV TEST POLY PRED': raae_test_polynomial_pred,
                         'RAAE FV TEST LSTSQ': raae_test_lstsq,
                         'RMAE FV TEST PRED': rmae_test_pred,
                         'RMAE FV TEST POLY': rmae_test_polynomial,
                         'RMAE FV TEST POLY PRED': rmae_test_polynomial_pred,        
                         'RMAE FV TEST LSTSQ': rmae_test_lstsq,
                         'FD FV TEST PRED': fd_test_pred,    
                         'FD FV TEST POLY': fd_test_polynomial,    
                         'FD FV TEST POLY PRED': fd_test_polynomial_pred, 
                         'FD FV TEST LSTSQ': fd_test_lstsq,    
                         'DTW FV TEST PRED': dtw_test_pred,
                         'DTW FV TEST POLY': dtw_test_polynomial,
                         'DTW FV TEST POLY PRED': dtw_test_polynomial_pred, 
                         'DTW FV TEST LSTSQ': dtw_test_lstsq,
        },{
                         'STD FV TRAIN REAL': std_train_real, 
                         'STD FV VALID REAL': std_valid_real, 
                         'STD FV VALID PRED': std_valid_pred,
                         'STD FV VALID POLY': std_valid_polynomial, 
                         'STD FV VALID LSTSQ': std_valid_lstsq, 
                         'STD FV TEST REAL': std_test_real,
                         'STD FV TEST PRED': std_test_pred, 
                         'STD FV TEST POLY': std_test_polynomial, 
                         'STD FV TEST LSTSQ': std_test_lstsq, 
        },{
                         'MEAN FV TRAIN REAL': mean_train_real, 
                         'MEAN FV VALID REAL': mean_valid_real, 
                         'MEAN FV VALID PRED': mean_valid_pred,
                         'MEAN FV VALID POLY': mean_valid_polynomial, 
                         'MEAN FV VALID LSTSQ': mean_valid_lstsq, 
                         'MEAN FV TEST REAL': mean_test_real,
                         'MEAN FV TEST PRED': mean_test_pred, 
                         'MEAN FV TEST POLY': mean_test_polynomial, 
                         'MEAN FV TEST LSTSQ': mean_test_lstsq, 
        }]        
                  
            result_dict_list.append(result_dict_list_single_epoch)
       

        
    if printing:        
        for i, (weights_for_epoch, polynomial_pred_for_epoch, polynomial_lstsq_true_for_epoch) in enumerate(zip(weights, polynomial_pred_list, polynomial_lstsq_true_list)):        
            if each_epochs_save != None:
                path = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str((i+1)*each_epochs_save).zfill(3)  + filename + '.txt'
            else:
                path = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
            with open(path, 'a') as text_file: 
                for i, value in enumerate(polynomial.values):
                    if i == 0:
                        text_file.write(str(value))  
                    else:
                        text_file.write(', ' + str(value))   
                for value in polynomial_pred_for_epoch:
                    text_file.write(', ' + str(value))
                for value in polynomial_lstsq_true_for_epoch:
                    text_file.write(', ' + str(value))
                for layer_weights, biases in pairwise(weights_for_epoch):    #clf.get_weights()
                    for neuron in layer_weights:
                        for weight in neuron:
                            text_file.write(', ' + str(weight))
                    for bias in biases:
                        text_file.write(', ' + str(bias))
                text_file.write('\n')

            text_file.close() 

            
    if return_model:
        return (polynomial, polynomial_pred_list, polynomial_lstsq_true_list), result_dict_list, pred_list, history, model
    elif return_history:
        return (polynomial, polynomial_pred_list, polynomial_lstsq_true_list), result_dict_list, pred_list, history, #polynomial_pred_list, polynomial_lstsq_true_list#, weights, history
    else:
        return (polynomial, polynomial_pred_list, polynomial_lstsq_true_list), result_dict_list, pred_list#, weights
    

In [23]:
clf_list = []
chunksize = 1000 if data_size > 10000 else max(data_size//10, min(50, data_size))
X_data_list_splits = list(gh.chunks(x_data, chunksize))
y_data_list_splits = list(gh.chunks(y_data, chunksize))

for X_data_list_split, y_data_list_split in tqdm(zip(X_data_list_splits, y_data_list_splits), total=max(len(X_data_list_splits), len(y_data_list_splits))):
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    clf_sublist = parallel(delayed(train_nn)(X_data, y_data, X_data[0], return_history=True, each_epochs_save=each_epochs_save, printing=True) for X_data, y_data in zip(X_data_list_split, y_data_list_split))  
    clf_list.extend(clf_sublist)
    del parallel
rand_index = np.random.randint(data_size)
random_network = train_nn(X_data_list[rand_index][1], y_data_list[rand_index][1], X_data_list[rand_index][0], callbacks=[PlotLossesCallback(skip_first=0)], return_model=True)


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.


NameError: name 'mean_absolute_percentage_error_keras' is not defined

## Save Lambda-Model scores

In [ ]:
if each_epochs_save == None:
    dict_list_valid = [clf[1][0] for clf in clf_list]
    dict_list_test = [clf[1][1] for clf in clf_list]
    dict_list_stds = [clf[1][2] for clf in clf_list]
    dict_list_means = [clf[1][3] for clf in clf_list]

    dict_list_valid_mean = pd.DataFrame(dict_list_valid, columns=dict_list_valid[0].keys()).mean()
    df_mean_scores_valid = pd.DataFrame(np.column_stack((dict_list_valid_mean.values[0::4], dict_list_valid_mean.values[1::4], dict_list_valid_mean.values[2::4], dict_list_valid_mean.values[3::4])), columns=['VALID PRED', 'VALID POLY', 'VALID POLY PRED', 'VALID LSTSQ'], index=['MAE FV', 'RMSE FV', 'MAPE FV', 'R2 FV', 'RAAE FV', 'RMAE FV', 'FD FV', 'DTW FV'])

    dict_list_test_mean = pd.DataFrame(dict_list_test, columns=dict_list_test[0].keys()).mean()
    df_mean_scores_test = pd.DataFrame(np.column_stack((dict_list_test_mean.values[0::4], dict_list_test_mean.values[1::4], dict_list_test_mean.values[2::4], dict_list_test_mean.values[3::4])), columns=['TEST PRED', 'TEST POLY', 'TEST POLY PRED', 'TEST LSTSQ'], index=['MAE FV', 'RMSE FV', 'MAPE FV', 'R2 FV', 'RAAE FV', 'RMAE FV', 'FD FV', 'DTW FV'])

    stds_mean = pd.DataFrame(dict_list_stds, columns=dict_list_stds[0].keys()).mean()
    df_stds = pd.DataFrame(stds_mean, columns=['0'], index=stds_mean.index)
    
    means_mean = pd.DataFrame(dict_list_means, columns=dict_list_means[0].keys()).mean()
    df_means = pd.DataFrame(means_mean, columns=['0'], index=means_mean.index)
    
    path_scores_valid = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/scores_valid_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
    path_scores_test = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/scores_test_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
    path_stds = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/stds_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
    path_means = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/means_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
   
    df_mean_scores_valid.to_csv(path_scores_valid, sep=',')
    df_mean_scores_test.to_csv(path_scores_test, sep=',')
    df_stds.to_csv(path_stds, sep=',')
    df_means.to_csv(path_means, sep=',')
else:
    scores_list = [clf[1] for clf in clf_list]
    
    scores_dict_list_by_epochs = [[] for i in range(epochs//each_epochs_save)]
    for scores_dict_list in scores_list:   
        for index, scores_dict in enumerate(scores_dict_list):
            scores_dict_list_by_epochs[index].append(scores_dict)
            
        
    for index, scores_dict_list_single_epoch in enumerate(scores_dict_list_by_epochs):
        index = (index+1)*each_epochs_save
        dict_list_valid = [dict_list[0] for dict_list in scores_dict_list_single_epoch]
        dict_list_test = [dict_list[1] for dict_list in scores_dict_list_single_epoch]
        dict_list_stds = [dict_list[2] for dict_list in scores_dict_list_single_epoch]
        dict_list_means = [dict_list[3] for dict_list in scores_dict_list_single_epoch]
        
        dict_list_valid_mean = pd.DataFrame(dict_list_valid, columns=dict_list_valid[0].keys()).mean()  
        dict_list_test_mean = pd.DataFrame(dict_list_test, columns=dict_list_test[0].keys()).mean()
        stds_mean = pd.DataFrame(dict_list_stds, columns=dict_list_stds[0].keys()).mean()
        means_mean = pd.DataFrame(dict_list_means, columns=dict_list_means[0].keys()).mean()

        if index == 1*each_epochs_save:
            df_mean_scores_valid = pd.DataFrame(np.column_stack((dict_list_valid_mean.values[0::4], dict_list_valid_mean.values[1::4], dict_list_valid_mean.values[2::4], dict_list_valid_mean.values[3::4])), columns=['VALID PRED E' + str(index), 'VALID POLY E' + str(index), 'VALID POLY PRED E' + str(index), 'VALID LSTSQ E' + str(index)], index=['MAE FV', 'RMSE FV', 'MAPE FV', 'R2 FV', 'RAAE FV', 'RMAE FV', 'FD FV', 'DTW FV'])
            df_mean_scores_test = pd.DataFrame(np.column_stack((dict_list_test_mean.values[0::4], dict_list_test_mean.values[1::4], dict_list_test_mean.values[2::4], dict_list_test_mean.values[3::4])), columns=['TEST PRED E' + str(index), 'TEST POLY E' + str(index), 'TEST POLY PRED E' + str(index), 'TEST LSTSQ E' + str(index)], index=['MAE FV', 'RMSE FV', 'MAPE FV', 'R2 FV', 'RAAE FV', 'RMAE FV', 'FD FV', 'DTW FV'])
            df_stds = pd.DataFrame(stds_mean, columns=['E' + str(index)], index=stds_mean.index)   
            df_means = pd.DataFrame(means_mean, columns=['E' + str(index)], index=means_mean.index)   
        else:
            df_mean_scores_valid_new = pd.DataFrame(np.column_stack((dict_list_valid_mean.values[0::4], dict_list_valid_mean.values[1::4], dict_list_valid_mean.values[2::4], dict_list_valid_mean.values[3::4])), columns=['VALID PRED E' + str(index), 'VALID POLY E' + str(index), 'VALID POLY PRED E' + str(index), 'VALID LSTSQ E' + str(index)], index=['MAE FV', 'RMSE FV', 'MAPE FV', 'R2 FV', 'RAAE FV', 'RMAE FV', 'FD FV', 'DTW FV'])
            df_mean_scores_valid = pd.concat([df_mean_scores_valid, df_mean_scores_valid_new],axis=1)  

            df_mean_scores_test_new = pd.DataFrame(np.column_stack((dict_list_test_mean.values[0::4], dict_list_test_mean.values[1::4], dict_list_test_mean.values[2::4], dict_list_test_mean.values[3::4])), columns=['TEST PRED E' + str(index), 'TEST POLY E' + str(index), 'TEST POLY PRED E' + str(index), 'TEST LSTSQ E' + str(index)], index=['MAE FV', 'RMSE FV', 'MAPE FV', 'R2 FV', 'RAAE FV', 'RMAE FV', 'FD FV', 'DTW FV'])
            df_mean_scores_test = pd.concat([df_mean_scores_test, df_mean_scores_test_new],axis=1)  

            df_stds_new = pd.DataFrame(stds_mean, columns=['E' + str(index)], index=stds_mean.index)
            df_stds = pd.concat([df_stds, df_stds_new],axis=1)  
            
            df_means_new = pd.DataFrame(means_mean, columns=['E' + str(index)], index=means_mean.index)
            df_means = pd.concat([df_means, df_means_new],axis=1)     

    path_scores_valid = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/scores_valid_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
    path_scores_test = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/scores_test_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
    path_stds = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/stds_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
    path_means = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/means_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'

    df_mean_scores_valid.to_csv(path_scores_valid, sep=',')
    df_mean_scores_test.to_csv(path_scores_test, sep=',')
    df_stds.to_csv(path_stds, sep=',')
    df_means.to_csv(path_means, sep=',')
        

In [ ]:
df_mean_scores_valid

In [ ]:
df_mean_scores_test

In [ ]:
df_stds

In [ ]:
df_means

In [ ]:
if each_epochs_save == None:
    polynomials = [clf[0][0] for clf in clf_list]
    polynomial_pred_lists = [clf[0][1] for clf in clf_list]
    polynomial_lstsq_true_lists = [clf[0][2] for clf in clf_list]
    
    polynomials_df = pd.DataFrame(polynomials)
    
    pred_evaluation_dataset_valid_real, pred_evaluation_dataset_valid, pred_evaluation_dataset_valid_polynomial, X_valid_data, pred_evaluation_dataset_test_real, pred_evaluation_dataset_test, pred_evaluation_dataset_test_polynomial, X_test_data = [clf[2] for clf in clf_list]

    pred_evaluation_dataset_valid_real = pred_evaluation_dataset_valid_real.ravel()
    pred_evaluation_dataset_test_real = pred_evaluation_dataset_test_real.ravel()
    pred_evaluation_dataset_valid = pred_evaluation_dataset_valid.ravel()
    pred_evaluation_dataset_test = pred_evaluation_dataset_test.ravel()
    pred_evaluation_dataset_valid_polynomial = pred_evaluation_dataset_valid_polynomial.ravel()
    pred_evaluation_dataset_test_polynomial = pred_evaluation_dataset_test_polynomial.ravel()
    
    pred_evaluation_dataset_valid_real_df = pd.DataFrame(pred_evaluation_dataset_valid_real, columns=[str(valid_data) for valid_data in X_valid_data])
    pred_evaluation_dataset_test_real_df = pd.DataFrame(pred_evaluation_dataset_test_real, columns=[str(test_data) for test_data in X_test_data])
    pred_evaluation_dataset_valid_df = pd.DataFrame(pred_evaluation_dataset_valid, columns=[str(valid_data) for valid_data in X_valid_data])
    pred_evaluation_dataset_test_df = pd.DataFrame(pred_evaluation_dataset_test, columns=[str(test_data) for test_data in X_test_data])
    pred_evaluation_dataset_valid_polynomial_df = pd.DataFrame(pred_evaluation_dataset_valid_polynomial, columns=[str(valid_data) for valid_data in X_valid_data])
    pred_evaluation_dataset_test_polynomial_df = pd.DataFrame(pred_evaluation_dataset_test_polynomial, columns=[str(test_data) for test_data in X_test_data])    
    
    pred_evaluation_dataset_valid_real_df = pd.concat([polynomials_df, pred_evaluation_dataset_valid_real_df], axis=1)
    pred_evaluation_dataset_test_real_df = pd.concat([polynomials_df, pred_evaluation_dataset_test_real_df], axis=1)
    pred_evaluation_dataset_valid_df = pd.concat([polynomials_df, pred_evaluation_dataset_valid_df], axis=1)
    pred_evaluation_dataset_test_df = pd.concat([polynomials_df, pred_evaluation_dataset_test_df], axis=1)
    pred_evaluation_dataset_valid_polynomial_df = pd.concat([polynomials_df, pred_evaluation_dataset_valid_polynomial_df], axis=1)
    pred_evaluation_dataset_test_polynomial_df = pd.concat([polynomials_df, pred_evaluation_dataset_test_polynomial_df], axis=1)
       
    path_pred_evaluation_dataset_valid_real = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/pred_evaluation_dataset_valid_real_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
    path_pred_evaluation_dataset_test_real = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/pred_evaluation_dataset_test_real_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
    path_pred_evaluation_dataset_valid = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/pred_evaluation_dataset_valid_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
    path_pred_evaluation_dataset_test = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/pred_evaluation_dataset_test_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
    path_pred_evaluation_dataset_valid_polynomial = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/pred_evaluation_dataset_valid_polynomial_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
    path_pred_evaluation_dataset_test_polynomial = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/pred_evaluation_dataset_test_polynomial_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
   
    pred_evaluation_dataset_valid_real_df.to_csv(path_pred_evaluation_dataset_valid_real, sep=',', index=False)
    pred_evaluation_dataset_test_real_df.to_csv(path_pred_evaluation_dataset_test_real, sep=',', index=False)
    pred_evaluation_dataset_valid_df.to_csv(path_pred_evaluation_dataset_valid, sep=',', index=False)
    pred_evaluation_dataset_test_df.to_csv(path_pred_evaluation_dataset_test, sep=',', index=False)
    pred_evaluation_dataset_valid_polynomial_df.to_csv(path_pred_evaluation_dataset_valid_polynomial, sep=',', index=False)
    pred_evaluation_dataset_test_polynomial_df.to_csv(path_pred_evaluation_dataset_test_polynomial, sep=',', index=False)
    
else:
    
    polynomials = [clf[0][0] for clf in clf_list]
    polynomial_pred_lists = [clf[0][1] for clf in clf_list]
    polynomial_lstsq_true_lists = [clf[0][2] for clf in clf_list]
    
    polynomials_df = pd.DataFrame(polynomials)
    
    pred_evaluation_dataset_valid_real_list = [np.zeros((len(clf_list), len(clf_list[0][2][0][0]))) for i in range(epochs//each_epochs_save)]
    pred_evaluation_dataset_valid_list = [np.zeros((len(clf_list), len(clf_list[0][2][0][1]))) for i in range(epochs//each_epochs_save)]
    pred_evaluation_dataset_valid_polynomial_list = [np.zeros((len(clf_list), len(clf_list[0][2][0][2]))) for i in range(epochs//each_epochs_save)]
    pred_evaluation_dataset_test_real_list = [np.zeros((len(clf_list), len(clf_list[0][2][0][4]))) for i in range(epochs//each_epochs_save)]
    pred_evaluation_dataset_test_list = [np.zeros((len(clf_list), len(clf_list[0][2][0][5]))) for i in range(epochs//each_epochs_save)]
    pred_evaluation_dataset_test_polynomial_list = [np.zeros((len(clf_list), len(clf_list[0][2][0][6]))) for i in range(epochs//each_epochs_save)]
    
    for i, pred_evaluation_dataset_per_epoch in tqdm(enumerate([clf[2] for clf in clf_list]), total=len(clf_list)):
        
        for index, (pred_evaluation_dataset_valid_real, pred_evaluation_dataset_valid, pred_evaluation_dataset_valid_polynomial, X_valid_data, pred_evaluation_dataset_test_real, pred_evaluation_dataset_test, pred_evaluation_dataset_test_polynomial, X_test_data) in enumerate(pred_evaluation_dataset_per_epoch):
            pred_evaluation_dataset_valid_real_list[index][i] = pred_evaluation_dataset_valid_real.ravel()
            pred_evaluation_dataset_valid_list[index][i] = pred_evaluation_dataset_valid.ravel()
            pred_evaluation_dataset_valid_polynomial_list[index][i] = pred_evaluation_dataset_valid_polynomial.ravel()
            
            pred_evaluation_dataset_test_real_list[index][i] = pred_evaluation_dataset_test_real.ravel()
            pred_evaluation_dataset_test_list[index][i] = pred_evaluation_dataset_test.ravel()
            pred_evaluation_dataset_test_polynomial_list[index][i] = pred_evaluation_dataset_test_polynomial.ravel()
    
    for index, (pred_evaluation_dataset_valid_real_by_epoch, pred_evaluation_dataset_valid_by_epoch, pred_evaluation_dataset_valid_polynomial_by_epoch, pred_evaluation_dataset_test_real_by_epoch, pred_evaluation_dataset_test_by_epoch, pred_evaluation_dataset_test_polynomial_by_epoch) in tqdm(enumerate(zip(pred_evaluation_dataset_valid_real_list, pred_evaluation_dataset_valid_list, pred_evaluation_dataset_valid_polynomial_list, pred_evaluation_dataset_test_real_list, pred_evaluation_dataset_test_list, pred_evaluation_dataset_test_polynomial_list)), total=len(pred_evaluation_dataset_valid_list)):
        pred_evaluation_dataset_valid_real_df = pd.DataFrame(pred_evaluation_dataset_valid_real_by_epoch, columns=[str(valid_data) for valid_data in X_valid_data])
        pred_evaluation_dataset_valid_df = pd.DataFrame(pred_evaluation_dataset_valid_by_epoch, columns=[str(valid_data) for valid_data in X_valid_data])
        pred_evaluation_dataset_valid_polynomial_df = pd.DataFrame(pred_evaluation_dataset_valid_polynomial_by_epoch, columns=[str(valid_data) for valid_data in X_valid_data])  
        pred_evaluation_dataset_test_real_df = pd.DataFrame(pred_evaluation_dataset_test_real_by_epoch, columns=[str(test_data) for test_data in X_test_data])
        pred_evaluation_dataset_test_df = pd.DataFrame(pred_evaluation_dataset_test_by_epoch, columns=[str(test_data) for test_data in X_test_data])
        pred_evaluation_dataset_test_polynomial_df = pd.DataFrame(pred_evaluation_dataset_test_polynomial_by_epoch, columns=[str(test_data) for test_data in X_test_data])   
        
        pred_evaluation_dataset_valid_real_df = pd.concat([polynomials_df, pred_evaluation_dataset_valid_real_df], axis=1)
        pred_evaluation_dataset_test_real_df = pd.concat([polynomials_df, pred_evaluation_dataset_test_real_df], axis=1)
        pred_evaluation_dataset_valid_df = pd.concat([polynomials_df, pred_evaluation_dataset_valid_df], axis=1)
        pred_evaluation_dataset_test_df = pd.concat([polynomials_df, pred_evaluation_dataset_test_df], axis=1)
        pred_evaluation_dataset_valid_polynomial_df = pd.concat([polynomials_df, pred_evaluation_dataset_valid_polynomial_df], axis=1)
        pred_evaluation_dataset_test_polynomial_df = pd.concat([polynomials_df, pred_evaluation_dataset_test_polynomial_df], axis=1)

        path_pred_evaluation_dataset_valid_real = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/pred_evaluation_dataset_valid_real_' + 'epoch_' + str(index).zfill(3) + '_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
        path_pred_evaluation_dataset_test_real = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/pred_evaluation_dataset_test_real_' + 'epoch_' + str(index).zfill(3) + '_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
        path_pred_evaluation_dataset_valid = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/pred_evaluation_dataset_valid_' + 'epoch_' + str(index).zfill(3) + '_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
        path_pred_evaluation_dataset_test = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/pred_evaluation_dataset_test_' + 'epoch_' + str(index).zfill(3) + '_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
        path_pred_evaluation_dataset_valid_polynomial = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/pred_evaluation_dataset_valid_polynomial_' + 'epoch_' + str(index).zfill(3) + '_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
        path_pred_evaluation_dataset_test_polynomial = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/pred_evaluation_dataset_test_polynomial_' + 'epoch_' + str(index).zfill(3) + '_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'

        pred_evaluation_dataset_valid_real_df.to_csv(path_pred_evaluation_dataset_valid_real, sep=',', index=False)
        pred_evaluation_dataset_test_real_df.to_csv(path_pred_evaluation_dataset_test_real, sep=',', index=False)         
        pred_evaluation_dataset_valid_df.to_csv(path_pred_evaluation_dataset_valid, sep=',', index=False)
        pred_evaluation_dataset_test_df.to_csv(path_pred_evaluation_dataset_test, sep=',', index=False)    
        pred_evaluation_dataset_valid_polynomial_df.to_csv(path_pred_evaluation_dataset_valid_polynomial, sep=',', index=False)
        pred_evaluation_dataset_test_polynomial_df.to_csv(path_pred_evaluation_dataset_test_polynomial, sep=',', index=False) 


In [ ]:
pred_evaluation_dataset_test_real_df.head()

In [ ]:
pred_evaluation_dataset_test_df.head()

In [ ]:
pred_evaluation_dataset_test_polynomial_df.head()

In [ ]:
#variable_values = []
#for column in pred_evaluation_dataset_df.columns:
#    variable_values.append(np.array(column[1:-1].split()).astype('float'))
#variable_values = np.array(variable_values)

In [ ]:
#fv_with_vv = []
#for function_values in tqdm(pred_evaluation_dataset_df.values):
#    fv_with_vv.append(np.array([np.append(vv, fv) for fv, vv in zip(function_values, variable_values)]))
#fv_with_vv = np.array(fv_with_vv)

In [ ]:
loss_list_total = []
metric_list_total = []

val_loss_list_total = []
val_metric_list_total = []

for _, entry in tqdm(enumerate(clf_list)):
    entry = entry[3]
    loss_list = []
    metric_list = []
    val_loss_list = []
    val_metric_list = []
    for i in range(epochs):  
        loss_list.append(entry['loss'][i])
        metric_list.append(entry['mean_absolute_percentage_error_keras'][i])
        val_loss_list.append(entry['val_loss'][i])
        val_metric_list.append(entry['val_mean_absolute_percentage_error_keras'][i])
    loss_list_total.append(loss_list)
    metric_list_total.append(metric_list)
    val_loss_list_total.append(val_loss_list)
    val_metric_list_total.append(val_metric_list)

loss_df = pd.DataFrame(data=loss_list_total, columns=['loss_epoch_' + str(i+1) for i in range(epochs)])
metric_df = pd.DataFrame(data=metric_list_total, columns=['metric_epoch_' + str(i+1) for i in range(epochs)]) 
val_loss_df = pd.DataFrame(data=val_loss_list_total, columns=['val_loss_epoch_' + str(i+1) for i in range(epochs)])
val_metric_df = pd.DataFrame(data=val_metric_list_total, columns=['val_metric_epoch_' + str(i+1) for i in range(epochs)]) 


In [ ]:
path_loss = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/history_loss_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
path_metric = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/history_metric_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
path_val_loss = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/history_val_loss_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'
path_val_metric = './data/weights/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/history_val_metric_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.txt'

loss_df.to_csv(path_loss, index=None, sep=',')
metric_df.to_csv(path_metric, index=None, sep=',')
val_loss_df.to_csv(path_val_loss, index=None, sep=',')
val_metric_df.to_csv(path_val_metric, index=None, sep=',')


In [ ]:
loss_df.describe()

In [ ]:
val_loss_df.describe()

In [ ]:
metric_df.describe()

In [ ]:
val_metric_df.describe()

In [ ]:
# summarize history for metric
path = './data/results/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/metric_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.png'

adjustment_threshold_metric = 0#100
    
metric_df_adjusted = metric_df.copy(deep=True)
if adjustment_threshold_metric > 0:
    metric_df_adjusted[metric_df_adjusted.columns] = np.where(metric_df_adjusted[metric_df_adjusted.columns] > adjustment_threshold_metric, adjustment_threshold_metric, metric_df_adjusted[metric_df_adjusted.columns])
    
val_metric_df_adjusted = val_metric_df.copy(deep=True)
if adjustment_threshold_metric > 0:
    val_metric_df_adjusted[val_metric_df_adjusted.columns] = np.where(val_metric_df_adjusted[val_metric_df_adjusted.columns] > adjustment_threshold_metric, adjustment_threshold_metric, val_metric_df_adjusted[val_metric_df_adjusted.columns])

    
plt.plot(metric_df_adjusted.describe().loc['mean'].values)
plt.plot(val_metric_df_adjusted.describe().loc['mean'].values)
plt.title('model metric')
plt.ylabel('metric')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(path)
plt.show()


In [ ]:
# summarize history for loss
path = './data/results/weights_' + str(data_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + filename + '/loss_' + str(data_size) +  '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_amax_' + str(a_max) + '_xmax_' + str(x_max) + training_string + '_epoch_' + str(epochs).zfill(3)  + filename + '.png'

adjustment_threshold_loss = 0#10000
    
loss_df_adjusted = loss_df.copy(deep=True)

if adjustment_threshold_loss > 0:
    loss_df_adjusted[loss_df_adjusted.columns] = np.where(loss_df_adjusted[loss_df_adjusted.columns] > adjustment_threshold_loss, adjustment_threshold_loss, loss_df_adjusted[loss_df_adjusted.columns])
    
val_loss_df_adjusted = val_loss_df.copy(deep=True)
if adjustment_threshold_loss > 0:
    val_loss_df_adjusted[val_loss_df_adjusted.columns] = np.where(val_loss_df_adjusted[val_loss_df_adjusted.columns] > adjustment_threshold_loss, adjustment_threshold_loss, val_loss_df_adjusted[val_loss_df_adjusted.columns])

    
plt.plot(loss_df_adjusted.describe().loc['mean'].values)
plt.plot(val_loss_df_adjusted.describe().loc['mean'].values)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.savefig(path)
plt.show()
